#  Text2SQL via Prompt Engineering

## **Retrieve Data**

In [1]:
!curl "https://api.mockaroo.com/api/5991a870?count=1000&key=94dad050" > "department.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60684    0 60684    0     0  42186      0 --:--:--  0:00:01 --:--:-- 42170


In [2]:
!curl "https://api.mockaroo.com/api/ccf77190?count=1000&key=94dad050" > "employee.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  101k    0  101k    0     0  37796      0 --:--:--  0:00:02 --:--:-- 37790


## **Setup Database**

In [3]:
import sqlite3
import pandas as pd
import os

In [4]:
# Define SQL schemas for creating tables
department_schema="""
create table department (
	department_name VARCHAR(50),
	department_code VARCHAR(10),
	department_head VARCHAR(50),
	department_budget VARCHAR(50),
	department_location VARCHAR(50)
);
"""

employee_schema="""
create table employee (
	employee_id INT,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	gender VARCHAR(50),
	date_of_birth DATE,
	job_title VARCHAR(50),
	department VARCHAR(50),
	salary DECIMAL(8,2),
	hire_date DATE,
	email VARCHAR(50)
);
"""

In [7]:
db_name = 'org.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [8]:
import sqlite3
import pandas as pd
import os

COLUMN_DATA_TYPES = {
    'department': {
        'department_name': 'object',
	      'department_code': 'object',
	      'department_head': 'object',
	      'department_budget': 'object',
	      'department_location': 'object'
    },

    'employee': {
        'employee_id': 'int64',
	      'first_name': 'object',
	      'last_name': 'object',
	      'gender': 'object',
	      'date_of_birth': 'datetime64[ns]',
	      'job_title': 'object',
	      'department': 'object',
	      'salary': 'float64',
	      'hire_date': 'datetime',
        'email':'object'
        }
    }


In [9]:
# --- Database setup ---
db_name = 'org.db'
conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables
    cursor.execute(department_schema)
    cursor.execute(employee_schema)

    print("Tables 'deaprtment' and 'employee' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/department.csv': 'department',
        '/content/employee.csv': 'employee'
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # # 1. Get the expected schema for the current table
            # expected_schema = COLUMN_DATA_TYPES[table_name]
            # expected_cols = list(expected_schema.keys())

            # # 2. Handle missing/extra columns
            # # Drop columns from DataFrame that are not in the schema
            # df = df[df.columns.intersection(expected_cols)]

            # # Add any missing columns and fill with None (which becomes NULL in SQL)
            # for col in expected_cols:
            #     if col not in df.columns:
            #         df[col] = None

            # # 3. Reorder columns to match the defined schema exactly
            # df = df[expected_cols]

            # # 4. Enforce data types
            # for col, dtype in expected_schema.items():
            #     if 'datetime' in dtype:
            #         # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
            #         df[col] = pd.to_datetime(df[col], errors='coerce')
            #     else:
            #         # Use astype for other columns, handling potential conversion errors
            #         try:
            #             df[col] = df[col].astype(dtype)
            #         except (ValueError, TypeError) as e:
            #             print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


            # Use the to_sql method to insert the cleaned DataFrame
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")

Database 'org.db' created and connected successfully. ✅
Tables 'deaprtment' and 'employee' created successfully.

Processing '/content/department.csv' for table 'department'...
  -> Data from '/content/department.csv' loaded into 'department' table successfully.

Processing '/content/employee.csv' for table 'employee'...
  -> Data from '/content/employee.csv' loaded into 'employee' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


## **Setup API Key using Google's AI Studio**

https://aistudio.google.com/

In [10]:
#Install GenAI library
!pip install google-genai

In [11]:
from google import genai
from google.colab import userdata

In [20]:
genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

## **Prompt Engineering**

In [14]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's org database which contains employee and department tables using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following two tables:

**`department` table:**

```sql
create table department (
	department_name VARCHAR(50),
	department_code VARCHAR(10),
	department_head VARCHAR(50),
	department_budget VARCHAR(50),
	department_location VARCHAR(50)
);
```

**`employee` table:**

```sql
create table employee (
	employee_id INT,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	gender VARCHAR(50),
	date_of_birth DATE,
	job_title VARCHAR(50),
	department VARCHAR(50),
	salary DECIMAL(8,2),
	hire_date DATE,
	email VARCHAR(50)
);
```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`department`, `employee`) and columns. Determine the necessary `JOIN` operations using `employee.employee_id` and `department.department_code` as foreign key in the `employee` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **CONSTRAINTS**

  * **Read-Only Operations:** You must **ONLY** generate `SELECT` queries. Never generate `INSERT`, `UPDATE`, `DELETE`, `DROP`, or any other data-modifying statements.
  * **Adhere Strictly to Schema:** Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.
  * **No Explanations:** Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
  * **Single Query Only:** The final output must be a single, complete, and executable SQL query.
  * **Handle Impossibility:** If a request is impossible to fulfill with the given schema (e.g., "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### **EXAMPLES**

**Example 1: Simple Lookup**

  * **User Query:** "Show me all employees in the Sales department."
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT employee_id, first_name, last_name, job_title, department FROM employee WHERE department = 'Sales';"
    }
    ```

**Example 2: Complex Join and Aggregation**

  * **User Query:** "Find the names and salaries of employees who earn more than 80,000."
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT first_name, last_name, salary FROM employee WHERE salary > 80000;"
    }
    ```

**Example 3: Ambiguous Query**

  * **User Query:** "Find employees with a high salary."
  * **Expected Output:**
    ```json
    {
      "status": "clarification_needed",
      "response": "Could you specify what ‘high salary’ means — a specific threshold (e.g., >80,000) or relative (e.g., top 10% earners)?"
    }
    ```

**Example 4: Impossible Query**

  * **User Query:** "Show me the performance ratings of all employees"
  * **Expected Output:**
    ```json
    {
      "status": "error",
      "response": "This request cannot be answered because the schema does not include a column for performance ratings. Available employee-related fields are: employee_id, first_name, last_name, gender, date_of_birth, job_title, department, salary, hire_date, email."
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated.
"""

In [15]:
import json
def get_sql_query(genai_client, prompt, user_query):

  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output

In [16]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='org.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [17]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [21]:
text2sql(genai_client, prompt, "Show me total count of employees in each department")

Input Token Count: 1321
Thoughts Token Count: 58
Output Token Count: 41
Total Token Count: 1420

Executing query on 'org.db':
SELECT department, COUNT(employee_id) AS total_employees FROM employee GROUP BY department;
Query executed successfully.


,department,total_employees
0,Finance,190
1,HR,199
2,IT,235
3,Marketing,184
4,Operations,192


In [23]:
text2sql(genai_client, prompt, "Show me all employees in the IT department.")

Input Token Count: 1321
Thoughts Token Count: 238
Output Token Count: 48
Total Token Count: 1607

Executing query on 'org.db':
SELECT employee_id, first_name, last_name, job_title, department FROM employee WHERE department = 'IT';
Query executed successfully.


,employee_id,first_name,last_name,job_title,department
0,3,Buffy,Kopf,Operator,IT
1,4,Sharl,Van der Krui,Clinical Specialist,IT
2,9,Corey,Barajaz,Safety Technician I,IT
3,20,Maddalena,Eul,Editor,IT
4,28,Wiley,Verrier,VP Accounting,IT
...,...,...,...,...,...
230,980,Cristin,McVanamy,Accountant II,IT
231,982,Shaun,Warbrick,Quality Engineer,IT
232,988,Bertie,Hearnden,Developer III,IT
233,989,Abbott,Teeney,VP Sales,IT


In [27]:
text2sql(genai_client, prompt, "Find the names and salaries of employees who earn more than 80,000.")

Input Token Count: 1331
Thoughts Token Count: 98
Output Token Count: 44
Total Token Count: 1473

Executing query on 'org.db':
SELECT first_name, last_name, salary FROM employee WHERE salary > 80000;
Query executed successfully.


,first_name,last_name,salary
0,Marwin,Coste,93163.10
1,Lukas,Buddleigh,139302.45
2,Gregoire,Heaviside,140331.93
3,Reinaldos,Lount,137760.57
4,Corey,Barajaz,144797.46
...,...,...,...
577,Salvador,Batchan,133686.70
578,Emilee,Matejic,98095.70
579,Elianora,Christon,114843.43
580,Devin,Gorthy,80181.21


In [28]:
text2sql(genai_client, prompt, "List the department name and head for all departments located in New York.")

Input Token Count: 1326
Thoughts Token Count: 170
Output Token Count: 41
Total Token Count: 1537

Executing query on 'org.db':
SELECT department_name, department_head FROM department WHERE department_location = 'New York';
Query executed successfully.


""
